In [87]:
from binarytree import Node
from collections import namedtuple

num_nodes = 15
global_node_list = [Node(chr(ord('A') + i)) for i in range(num_nodes)]

def print_nodes_list(view, nodes_list):
    print(f"{view}: {[(i, n.value) for i, n in enumerate(nodes_list)]}")

def nodes_list_to_map(l):
    l.sort(key=lambda x: x.value)
    return {i: n for i, n in enumerate(l)}

def init_node(node, nodes_list, layer):
    if node is None:
        return

    print_nodes_list(node.value, nodes_list)

    if layer == 0:
        return

    if len(nodes_list) == 0 or node.left is not None or node.right is not None:
        return

    nodes_list.sort(key=lambda x: x.value)

    i = nodes_list.index(node)
    li = int(i + len(nodes_list) * 1 / 3) % len(nodes_list)
    ri = int(i + len(nodes_list) * 2 / 3) % len(nodes_list)
    
    if i != li and (nodes_list[li].left is None and nodes_list[li].right is None):
        node.left = nodes_list[li]
    if i != ri and (nodes_list[ri].left is None and nodes_list[ri].right is None):
        node.right = nodes_list[ri]

    nodes_list = nodes_list[0:i] + nodes_list[i+1:]
    init_node(node.left, nodes_list.copy(), layer-1)
    init_node(node.right, nodes_list.copy(), layer-1)

print_nodes_list("global", global_node_list)

# originator_node = global_node_list[0]
# originator_node_list = global_node_list[1:]
# init_node(originator_node, originator_node_list, 3)
init_node(global_node_list[0], global_node_list, 3)

print(global_node_list[0])


global: [(0, 'A'), (1, 'B'), (2, 'C'), (3, 'D'), (4, 'E'), (5, 'F'), (6, 'G'), (7, 'H'), (8, 'I'), (9, 'J'), (10, 'K'), (11, 'L'), (12, 'M'), (13, 'N'), (14, 'O')]
A: [(0, 'A'), (1, 'B'), (2, 'C'), (3, 'D'), (4, 'E'), (5, 'F'), (6, 'G'), (7, 'H'), (8, 'I'), (9, 'J'), (10, 'K'), (11, 'L'), (12, 'M'), (13, 'N'), (14, 'O')]
F: [(0, 'B'), (1, 'C'), (2, 'D'), (3, 'E'), (4, 'F'), (5, 'G'), (6, 'H'), (7, 'I'), (8, 'J'), (9, 'K'), (10, 'L'), (11, 'M'), (12, 'N'), (13, 'O')]
J: [(0, 'B'), (1, 'C'), (2, 'D'), (3, 'E'), (4, 'G'), (5, 'H'), (6, 'I'), (7, 'J'), (8, 'K'), (9, 'L'), (10, 'M'), (11, 'N'), (12, 'O')]
N: [(0, 'B'), (1, 'C'), (2, 'D'), (3, 'E'), (4, 'G'), (5, 'H'), (6, 'I'), (7, 'K'), (8, 'L'), (9, 'M'), (10, 'N'), (11, 'O')]
D: [(0, 'B'), (1, 'C'), (2, 'D'), (3, 'E'), (4, 'G'), (5, 'H'), (6, 'I'), (7, 'K'), (8, 'L'), (9, 'M'), (10, 'N'), (11, 'O')]
O: [(0, 'B'), (1, 'C'), (2, 'D'), (3, 'E'), (4, 'G'), (5, 'H'), (6, 'I'), (7, 'J'), (8, 'K'), (9, 'L'), (10, 'M'), (11, 'N'), (12, 'O')]
E: 

In [186]:
# Assumption 1: Unique addresses that can be sorted lexicographically
# Assumption 2: Full view of the network (i.e. list to all nodes is available)

# Definition 1: Layer - opposite of tree depth (i.e. max # of layers = tree height)
# Definition 2: Global Address Book - list of all nodes in the network
# Definition 3: Partial Address Book - partial list of all nodes in the network (either due to algorithm or due to lack of information)

# Let X be the # of messages sent by each node (e.g. binary tree => 2; ternary tree => 3)
# Let Y be the target % coverage of of each node (e.g. 2/3 means node aims to propagate message to ~66% of the network)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import random
import heapq
from binarytree import Node, tree, bst, heap, build
from collections import defaultdict

global_counter_send = 0
global_set_reached = set()
global_map_received = defaultdict(int)

X = 2 # This is not parameterized but manually implemented
Y = 2/3

num_nodes = 6
nodes_addr = [chr(ord('A') + i) for i in range(num_nodes)]
nodes_addr.sort()

def get_subset(l, i1, i2):
    if i1 <= i2:
        return set(l[i1:i2 + 1])
    return set(l[i1:] + l[:i2])

def prop(addr, book):
    global global_counter_send
    global global_set_reached
    global global_map_received

    if addr == None:
        return

    global_set_reached.add(addr)

    print('-----')
    if len(book) <= 1:
        print(f"{addr} - {book}")
        print("No propagation")
        return

    n = len(book)
    i = book.index(addr)
    s = n * Y

    r1 = (i % n, (i + s - 1) % n)
    r2 = ((i + s) % n, (i + s + s) % n)

    r1_set = get_subset(book, int(r1[0]), int(r1[1]))
    r2_set = get_subset(book, int(r2[0]), int(r2[1])) 

    r1_set.discard(addr)
    r2_set.discard(addr)

    bookp = set(book)
    t1, t2 = None, None

    if len(r1_set) > 0:
        t1 = random.sample(r1_set, 1)[0]
        bookp.discard(t1)
    if len(r2_set) > 0:
        t2 = random.sample(r2_set, 1)[0]
        bookp.discard(t2)

    heapq.heapify(book)
    root = build(book)
    print(root)
    print(f"{addr} - {book}")    
    print(f"R1: {r1} : {r1_set} => sending to {t1}")
    print(f"R2: {r2} : {r2_set} => sending to {t2}")

    if t1 is not None:
        global_counter_send += 1
        prop(t1, list(r1_set))
    if t2 is not None:
        global_counter_send += 1
        prop(t2, list(r2_set))
    prop(addr, list(bookp))

print(f"Target Coverage: {Y}")
prop(nodes_addr[0], nodes_addr)
print(f"Global Send Counter: {global_counter_send}")
print(f"Global Set Reached: {sorted(list(global_set_reached))}")
print(f"Nodes not reached: {global_set_reached.difference(nodes_addr)}")

Target Coverage: 0.6666666666666666
-----

    __A__
   /     \
  B       C
 / \     /
D   E   F

A - ['A', 'B', 'C', 'D', 'E', 'F']
R1: (0, 3.0) : {'D', 'B', 'C'} => sending to D
R2: (4.0, 2.0) : {'F', 'B', 'E'} => sending to F
-----

  B
 / \
D   C

D - ['B', 'D', 'C']
R1: (0, 1.0) : {'B'} => sending to B
R2: (2.0, 1.0) : {'C'} => sending to C
-----
B - ['B']
No propagation
-----
C - ['C']
No propagation
-----
D - ['D']
No propagation
-----

  B
 / \
F   E

F - ['B', 'F', 'E']
R1: (0, 1.0) : {'B'} => sending to B
R2: (2.0, 1.0) : {'E'} => sending to E
-----
B - ['B']
No propagation
-----
E - ['E']
No propagation
-----
F - ['F']
No propagation
-----

    A
   / \
  C   B
 /
E

A - ['A', 'C', 'B', 'E']
R1: (1, 2.6666666666666665) : {'B'} => sending to B
R2: (3.6666666666666665, 2.333333333333333) : {'C', 'E'} => sending to C
-----
B - ['B']
No propagation
-----

  C
 /
E

C - ['C', 'E']
R1: (0, 0.33333333333333326) : set() => sending to None
R2: (1.3333333333333333, 0.6666666666666665)

In [114]:
l  = [1,2,3,4,5,6]
l[5:1]

[]